<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

# Hierarchical Indexing,层级索引

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects, respectively.


Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.


While Pandas does provide ``Panel`` and ``Panel4D`` objects that natively handle three-dimensional and four-dimensional data (see [Aside: Panel Data](#Aside:-Panel-Data)), a far more common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.

三维/4维数组

In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In this section, we'll explore the direct creation of ``MultiIndex`` objects, considerations when indexing, slicing, and computing statistics across multiply indexed data, and useful routines for converting between simple and hierarchically indexed representations of your data.


We begin with the standard imports:

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional ``Series``.

使用Series表示2维数据

For concreteness, we will consider a series of data where each point has a character and numerical key.

### The bad way, 一个笨办法

Suppose you would like to track data about states from two different years.
Using the Pandas tools we've already covered,
you might be tempted to simply use Python tuples as keys: 使用tuple当作key

In [5]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]

# tuple-list 

populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]


pop = pd.Series(populations, index=index)  # data, index

pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [7]:
pop[('California', 2010):('Texas', 2000)]  # Series

pop.index

[x for x in pop.index if x[1] > 2000 ]

[('California', 2010), ('New York', 2010), ('Texas', 2010)]

But the convenience ends there.
方便到此位置
For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:
选择2010的值

In [8]:
pop[[i for i in pop.index if i[1] == 2010]]  # 查询索引,使用fancy-index

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

This produces the desired result, 
but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

### The Better Way: Pandas MultiIndex,  多个索引
Fortunately, Pandas provides a better way.

Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas ``MultiIndex`` type gives us the type of operations we wish to have.

We can create a multi-index from the tuples as follows:  多索引操作

In [13]:
index = pd.MultiIndex.from_tuples(index)  # MultiIndex,多极索引对象, 建立了多索引对象
index  # 是pd的对象, level, lables

#index.values  #还是tuple

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Notice that the ``MultiIndex`` contains multiple *levels* of indexing–in this case, the state names and the years,
level是索引的值
as well as multiple *labels* for each data point which encode these levels.
label是索引的encode

If we re-index our series with this ``MultiIndex``, we see the hierarchical representation of the data:

使用reindex重新建立索引

In [17]:
pop = pop.reindex(index)
pop  # 新的数据
#pop.index  # 新的索引

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Here the first two columns of the ``Series`` representation show the multiple index values, while the third column shows the data.

前两列代表多极索引, 后面代表值

Notice that some entries are missing in the first column:

in this multi-index representation, any blank entry indicates the same value as the line above it.

值为空表示跟前面的值一样

Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [24]:
pop[:, 2010]   # pop

pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

The result is a singly indexed array with just the keys we're interested in.
This syntax is much more convenient (and the operation is much more efficient!) than the home-spun tuple-based multi-indexing solution that we started with.

多级索引语法简单

We'll now further discuss this sort of indexing operation on hieararchically indexed data.

### MultiIndex as extra dimension, 

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.

DataFrame使用index,column表示数据

In fact, Pandas is built with this equivalence in mind. 
The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:  

unstack能将一个多级索引转化为简单的索引DataFrame

In [29]:
pop_df = pop.unstack()  # Series 转 DataFrame, 多级索引转
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, the ``stack()`` method provides the opposite operation:

In [30]:
pop_df.stack()  # stack 提供想反的方法

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Seeing this, you might wonder why would we would bother with hierarchical indexing at all.


The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional ``Series``, we can also use it to represent data of three or more dimensions in a ``Series`` or ``DataFrame``.

Each extra level in a multi-index represents an extra dimension of data; 
taking advantage of this property gives us much more flexibility in the types of data we can represent.
Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a ``MultiIndex`` this is as easy as adding another column to the ``DataFrame``:

In [36]:
pop_df = pd.DataFrame({'total': pop,  # Series 对象
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

pop_df.index   # 依然是多级索引

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In addition, all the ufuncs and other functionality discussed in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) work with hierarchical indices as well.


Here we compute the fraction of people under 18 by year, given the above data:

In [38]:
f_u18 = pop_df['under18'] / pop_df['total']

#f_u18
f_u18.unstack()  

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


This allows us to easily and quickly manipulate and explore even high-dimensional data.


更加容易操作高纬数组

## Methods of MultiIndex Creation

The most straightforward way to construct a multiply indexed ``Series`` or ``DataFrame`` is 
to simply pass a list of two or more index arrays to the constructor.

构造多维数组,index是多个索引的list

For example:

In [44]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], # index = [a,b] a是index1,b是index2
                  columns=['data1', 'data2'])

df

data1     data2
a 1  0.819098  0.819354
  2  0.979228  0.978893
b 1  0.970451  0.839132
  2  0.407027  0.832045

The work of creating the ``MultiIndex`` is done in the background.

MultiIndex是在后台创建的

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a ``MultiIndex`` by default:

padas能自动识别index是tuple的类型

In [46]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
s1 = pd.Series(data)
s1.unstack()   # 创建DataFrame

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Nevertheless, it is sometimes useful to explicitly create a ``MultiIndex``; we'll see a couple of these methods here.

创建MultiIndex的方法


### Explicit MultiIndex constructors,显示创建多index

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the ``pd.MultiIndex``.

使用对象创建

For example, as we did before, you can construct the ``MultiIndex`` from a simple list of arrays giving the index values within each level:


In [51]:
s1 = pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])  # 使用list, tuple的每个元素组成的list

s2 = pd.MultiIndex.from_tuples([('a',1),('a',2),('b',1),('b',2)])  # 使用tuple的list,  
s2

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can construct it from a list of tuples giving the multiple index values of each point:

In [52]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

You can even construct it from a Cartesian product of single indices:

索引的笛卡尔乘机

In [72]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]],names=['index1','index2'])  # from_arrays  = from_product

#pd.MultiIndex.from_product?

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['index1', 'index2'])

Similarly, you can construct the ``MultiIndex`` 
directly using its internal encoding by passing ``levels`` (a list of lists containing available index values for each level) 

+ levels 是索引的标签
and ``labels`` (a list of lists that reference these labels):

+ labels 是标签对应的值0/1

In [17]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``,

构造函数的index函数

or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

方法reindex的参数

### MultiIndex level names

Sometimes it is convenient to name the levels of the ``MultiIndex``.
This can be accomplished by passing the ``names`` argument to any of the above ``MultiIndex`` constructors,

or by setting the ``names`` attribute of the index after the fact:


In [75]:
pop.index.names = ['state', 'year']  # 设置标签的name, 通过index的属性设置
pop.index.names # 输出或设置

FrozenList(['state', 'year'])

With more involved datasets, this can be a useful way to keep track of the meaning of various index values.

### MultiIndex for columns,列使用多索引

In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.

DataFrame的列也可以是多列

Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [82]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# 创建列的方法与行的方法一样

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  35.8  52.0  36.6  62.0  37.4
     2      40.0  35.9  52.0  38.3  46.0  37.2
2014 1      34.0  37.9  35.0  35.9  39.0  38.8
     2      10.0  38.4  41.0  36.0  24.0  37.8

Here we see where the multi-indexing for both rows and columns can come in *very* handy. 非常的简单

This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number.

创建的4维数据

With this in place we can, for example, index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:

In [87]:
health_data['Guido']
health_data['Guido','HR']  # 多级索引的访问方法

year  visit
2013  1        52.0
      2        52.0
2014  1        35.0
      2        41.0
Name: (Guido, HR), dtype: float64

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

## Indexing and Slicing a MultiIndex,  多级索引的分片,

Indexing and slicing on a ``MultiIndex`` is designed to be intuitive, and it helps if you think about the indices as added dimensions.

多级索引很简单设计

We'll first look at indexing multiply indexed ``Series``, and then multiply-indexed ``DataFrame``s.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier:

多级索引的Series

In [93]:
pop.unstack()
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [91]:
pop['California', 2000]  # 使用多个索引
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

The ``MultiIndex`` also supports *partial indexing*, or indexing just one of the levels in the index.
The result is another ``Series``, with the lower-level indices maintained:

多级索引支持部分索引,只使用多级索引中的一部分

In [92]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

Partial slicing is available as well, as long as the ``MultiIndex`` is sorted (see discussion in [Sorted and Unsorted Indices](#Sorted-and-unsorted-indices)):

多级索引支持分片


In [95]:
pop.loc['California':'New York']
pop.iloc[0:2]   # 按顺序显示

state       year
California  2000    33871648
            2010    37253956
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:



In [96]:
#pop[:, 2000]

pop[:,:]  # 使用索引的切片

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Other types of indexing and selection (discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb)) work as well; 

for example, selection based on Boolean masks:

In [97]:
pop[pop > 22000000]   # 使用bool掩码

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

Selection based on fancy indexing also works:

In [99]:
pop[['California', 'Texas']]   # 使用fancy-index

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:


In [104]:
health_data  # 行列都是二级索引

health_data.index

health_data.columns


health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  35.8  52.0  36.6  62.0  37.4
     2      40.0  35.9  52.0  38.3  46.0  37.2
2014 1      34.0  37.9  35.0  35.9  39.0  38.8
     2      10.0  38.4  41.0  36.0  24.0  37.8

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.

For example, we can recover Guido's heart rate data with a simple operation:

在DataFrame中,columns是第一索引

In [29]:
health_data['Guido', 'HR']  #  第一索引

year  visit
2013  1        32.0
      2        50.0
2014  1        39.0
      2        48.0
Name: (Guido, HR), dtype: float64

Also, as with the single-index case, we can use the ``loc``, ``iloc``, and ``ix`` indexers introduced in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb). For example:

In [30]:
health_data.iloc[:2, :2]  # 2个

subject      Bob      
type          HR  Temp
year visit            
2013 1      31.0  38.7
     2      44.0  37.7

These indexers provide an array-like view of the underlying two-dimensional data, 

but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

loc/ iloc 可以使用tuple作为索引的值

In [105]:
health_data.loc[:, ('Bob', 'HR')]  #多级索引中使用tuple作为index

year  visit
2013  1        43.0
      2        40.0
2014  1        34.0
      2        10.0
Name: (Bob, HR), dtype: float64

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:


可以不是分片, 但不要在tuple中使用分片,有语法错误

In [106]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-106-fb34fa30ac09>, line 1)

You could get around this by building the desired slice explicitly using Python's built-in ``slice()`` function,

but a better way in this context is to use an ``IndexSlice`` object, which Pandas provides for precisely this situation.


可以使用python的内建函数,但是更好的方法是使用indexSlice对象,

For example:

In [108]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]   #  使用IndexSlice对对象进行分片索引

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,43.0,52.0,62.0
2014,1,34.0,35.0,39.0


In [110]:
ids = pd.IndexSlice
health_data.loc[ids[2013,:],ids['Bob',:]]   #ids是对象, 可以重复使用

subject      Bob      
type          HR  Temp
year visit            
2013 1      43.0  35.8
     2      40.0  35.9

There are so many ways to interact with data in multiply indexed ``Series`` and ``DataFrame``s, and as with many tools in this book the best way to become familiar with them is to try them out!

最好的办法是使用他们

## Rearranging Multi-Indices, 多级索引排序

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.

使用多级索引需要知道怎么转换数据

There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations.
 
We saw a brief example of this in the ``stack()`` and ``unstack()`` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.


### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

对于许多的多级索引,如果索引是没有排序的,使用分片是报错的

We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

创建一个没有字段序的多级索引数据级

In [158]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])   # a,c,b 不是字典序
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data   # 第一个索引是没有排序

MultiIndex(levels=[['a', 'b', 'c'], [1, 2]],
           labels=[[0, 0, 2, 2, 1, 1], [0, 1, 0, 1, 0, 1]],
           names=['char', 'int'])

If we try to take a partial slice of this index, it will result in an error:

In [124]:
data[:,1] # 这个不报错

try:
    data[:'b',1]      #  TypeErrror
except Exception as e:  
    print(type(e))
    print(e)

<class 'TypeError'>
(slice(None, 'b', None), 1)


In [118]:
try:
    data['a':'b']    # KeyError
except KeyError as e:
    print(type(e))
    print(e)
    

TypeError: (slice('a', 'b', None), 1)

Although it is not entirely clear from the error message,this is the result of the MultiIndex not being sorted.
索引没有排序倒置的

For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.
多级索引的很多操作需要索引排序

Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.

pd提供了2个方法


We'll use the simplest, ``sort_index()``, here:

In [156]:
data = data.sort_index()  # 对索引的排序, 返回排序的对象
data
#mi = pd.MultiIndex.from_product([['b','c','a'],[3,2,1]])
#mv = mi.sort_values()   #对索引的排序
#[x for x in mv ]

data.index  # 看不出来是不是已经排序

MultiIndex(levels=[['a', 'b', 'c'], [1, 2]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=['char', 'int'])

With the index sorted in this way, partial slicing will work as expected:

In [37]:
data['a':'b']   # 索引排序后可以使用分片

char  int
a     1      0.003001
      2      0.164974
b     1      0.001693
      2      0.526226
dtype: float64

### Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, 

把多级索引转为2为数据

optionally specifying the level to use:

In [171]:
pop.unstack()  # level 默认是-1

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [169]:
pop.unstack(level=0)  # level=0 是第一轴线

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [170]:
pop.unstack(level=1)  # level 是第一轴线

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


The opposite of ``unstack()`` is ``stack()``, which here can be used to recover the original series:

In [172]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.

行列转换操作: reset_index能够把索引的标签转化为列值

Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.

For clarity, we can optionally specify the name of the data for the column representation:

In [215]:
# pop 中,state, year 是多级索引的标签值

pop_flat = pop.reset_index(name='population') 
# reset_inde会把多级索引的标签设置为column名,原来索引对应的数据通过name指定列名
pop_flat.index  # 索引变成index

#popx = pop.reset_index()  # 没有指定值的列名,居然默认是0, level是用来设置索引的
#popx

RangeIndex(start=0, stop=6, step=1)

Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.

This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

通过set_index方法建立多级索引的DataFrame

In [217]:
p1 = pop_flat.set_index(['state','year'])  #DataFrame变成多级索引列, 指定columns列为变成索引
p1  #DataFrame 变成多级索引

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In practice, I find this type of reindexing to be one of the more useful patterns when encountering real-world datasets.

## Data Aggregations on Multi-Indices,  多级索引的数据聚合

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
聚合函数
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.
多级索引的数据,通过level设置聚合计算的方向
For example, let's return to our health data:

In [219]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      43.0  35.8  52.0  36.6  62.0  37.4
     2      40.0  35.9  52.0  38.3  46.0  37.2
2014 1      34.0  37.9  35.0  35.9  39.0  38.8
     2      10.0  38.4  41.0  36.0  24.0  37.8

Perhaps we'd like to average-out the measurements in the two visits each year.每年各项数据统计
We can do this by naming the index level we'd like to explore, in this case the year:

指定索引的名字:level = index_name

In [222]:
data_mean = health_data.mean(level='year') # 每年
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     41.5  35.85  52.0  37.45  54.0  37.3
2014     22.0  38.15  38.0  35.95  31.5  38.3

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:

In [225]:
data_mean.mean(axis=1, level='type')  # 列索引需要设置axix = 1是x轴线的方向, 因为axis = 0 是y轴线的方向
data_mean.mean(level = 'subject',axis=1)  # 索引 + 列 计算聚合值, 必须指定轴线方向,否则会

ValueError: level name subject is not the name of the index

Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits each year.


This syntax is actually a short cut to the ``GroupBy`` functionality, which we will discuss in [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb).
While this is a toy example, many real-world datasets have similar hierarchical structure.

## Aside: Panel Data

Pandas has a few other fundamental data structures that we have not yet discussed, namely the ``pd.Panel`` and ``pd.Panel4D`` objects.
These can be thought of, respectively, as three-dimensional and four-dimensional generalizations of the (one-dimensional) ``Series`` and (two-dimensional) ``DataFrame`` structures.
Once you are familiar with indexing and manipulation of data in a ``Series`` and ``DataFrame``, ``Panel`` and ``Panel4D`` are relatively straightforward to use.
In particular, the ``ix``, ``loc``, and ``iloc`` indexers discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) extend readily to these higher-dimensional structures.

We won't cover these panel structures further in this text, as I've found in the majority of cases that multi-indexing is a more useful and conceptually simpler representation for higher-dimensional data.
Additionally, panel data is fundamentally a dense data representation, while multi-indexing is fundamentally a sparse data representation.
As the number of dimensions increases, the dense representation can become very inefficient for the majority of real-world datasets.
For the occasional specialized application, however, these structures can be useful.
If you'd like to read more about the ``Panel`` and ``Panel4D`` structures, see the references listed in [Further Resources](03.13-Further-Resources.ipynb).

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >